# 06-functionize.ipynb

Processing includes:
- image registration with `phase_cross_correlation`
- liquid subtraction
- adaptive histogram equalization

In [1]:
use_napari = False
import imageio.v3 as iio
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
if use_napari:
    import napari
import numpy as np
from pathlib import Path
from skimage import exposure, filters, registration, util
# Local imports
import helpers
# Enable autoreload for local modules
%load_ext autoreload
%autoreload 2

In [2]:
def align_and_sub_liq(imgs, clip=[0.1, 99.9], eq_hist=True):
    # Median filter images before converting to float
    print('Applying median filter...')
    imgs_med = filters.median(imgs)
    # Convert image to float before calculations
    imgs_float = util.img_as_float(imgs_med)
    # Calculate max offset between first and last image
    print('Calculating max offset between first and last images...')
    offset, error, diffphase = registration.phase_cross_correlation(
            imgs_float[0, :, :], imgs_float[-1, :, :])
    max_offset_r = int(offset[0])
    max_offset_c = int(offset[1])
    # Calc liquid-subtracted images with offset/drift-correction
    imgs_crctd = np.zeros(
            (imgs_float.shape[0],
            imgs_float.shape[1] - abs(max_offset_r),
            imgs_float.shape[2] - abs(max_offset_c)))
    # Iterate through each image and perform subtraction adjusting for offset
    print('Aligning each image and subtracting liquid image...')
    for i in range(imgs_float.shape[0]):
        offset, error, diffphase = registration.phase_cross_correlation(
                imgs_float[0, :, :], imgs_float[i, :, :])
        offset_r = int(offset[0])
        offset_c = int(offset[1])
        img_liq = imgs_float[
                0,
                : imgs_float.shape[1] - abs(max_offset_r),
                : imgs_float.shape[2] - abs(max_offset_c)]
        img_i = imgs_float[
                i,
                abs(offset_r) : imgs.shape[1] - (abs(max_offset_r) - abs(offset_r)),
                abs(offset_c) : imgs.shape[2] - (abs(max_offset_c) - abs(offset_c))]
        imgs_crctd[i, :, :] = img_i - img_liq
    if clip is not None:
        print('Clipping highest and lowest intensities...')
        low, high = np.percentile(imgs_crctd, (clip[0], clip[1]))
        imgs_crctd = np.clip(imgs_crctd, low, high)
    if eq_hist:
        print('Equalizing histograms...')
        imgs_sub_0to1 = exposure.rescale_intensity(
                imgs_crctd, in_range='image', out_range=(0, 1))
        imgs_crctd = exposure.equalize_adapthist(imgs_sub_0to1)
    return imgs_crctd

Note: You may need to put an "r" before the image directory path if the string contains backslashes. Don't forget commas between parameters!

In [9]:
img_dir = Path(
        r'X:\Radiography Drive\2013-11_APS'
        r'\Clarke_2013_November\048_Sn27Bi_100')
img_start = 352
img_stop = 372
img_step = 1
imgs_raw = helpers.get_imgs(
    img_dir,
    img_start=img_start,
    img_stop=img_stop,
    img_step=img_step,
    print_nums=True
)
imgs_crctd = align_and_sub_liq(imgs_raw, clip=[0.1, 99.9], eq_hist=True)
if use_napari:
    # Open napari window
    viewer = napari.Viewer()
    viewer.add_image(imgs_raw, name=f'imgs-{img_start}-{img_stop}-{img_step}')
    viewer.add_image(imgs_crctd)

Loading 20 images...
Images loaded:
['0: 352', '1: 353', '2: 354', '3: 355', '4: 356', '5: 357', '6: 358', '7: 359', '8: 360', '9: 361', '10: 362', '11: 363', '12: 364', '13: 365', '14: 366', '15: 367', '16: 368', '17: 369', '18: 370', '19: 371']
Applying median filter...
Calculating max offset between first and last images...
Aligning each image and subtracting liquid image...
Clipping highest and lowest intensities...
Equalizing histograms...


In [14]:
if True:
    save_dir = Path(
        r'C:\Users\cgusb\Research\aps-directional'
        r'\results\048_Sn27Bi_100-352-372-1_med-filt_liq-sub_aligned_clip')
    # Image widths and heights must be multiple of 2 for mp4 compression,
    # adjust odd number width by excluding last column
    helpers.save_as_pngs(
            save_dir, imgs_crctd[:, :, :], scalebar_dict={
                    'dx' : 1.4, 'units' : "um", 'length_fraction' : 0.2,
                    'border_pad' : 0.5, 'location' : 'lower right'})

Saving images...
20 images saved to: C:\Users\cgusb\Research\aps-directional\results\048_Sn27Bi_100-352-372-1_med-filt_liq-sub_aligned_clip
